In [2]:
# =============================================================================
# PROJECT 2: Marketing Multi-Channel Attribution & Funnel Analysis
# =============================================================================
# Scenario: Ingesting daily/weekly CSV exports from various marketing platforms.
# Skills: ETL from multiple file sources, Data Modeling, Last-Touch Attribution,
#         Funnel Analysis, SQL, Pandas, Data Visualization (Plotly).
# =============================================================================

# Step 0: Setup and Data Simulation
# -----------------------------------------------------------------------------
# We simulate the CSV content that a marketing team would typically export.

import pandas as pd
import numpy as np
import io
import sqlite3
import plotly.express as px
import plotly.graph_objects as go

def generate_google_ads_csv():
    return """Date,Campaign,Impressions,Clicks,Cost
2023-07-15,Summer Sale 2023,10000,500,200.50
2023-07-15,Brand Awareness,50000,1500,450.00
2023-07-16,Summer Sale 2023,12000,650,250.75
2023-07-16,Brand Awareness,55000,1600,475.50
"""

def generate_facebook_ads_csv():
    return """Day,AdSet,Reach,LinkClicks,AmountSpent
2023-07-15,FB-Summer-Promo,25000,800,180.00
2023-07-15,FB-Brand-General,60000,2000,400.00
2023-07-16,FB-Summer-Promo,28000,950,210.00
2023-07-16,FB-Brand-General,62000,2100,420.00
"""

def generate_analytics_csv():
    return """Date,UserID,SessionSource,SessionMedium,Device
2023-07-14,user456,facebook,social,Mobile
2023-07-15,user123,google,cpc,Desktop
2023-07-15,user456,facebook,social,Mobile
2023-07-15,user789,(direct),(none),Desktop
2023-07-16,user123,promo_email,email,Desktop
2023-07-16,user456,google,cpc,Mobile
"""

def generate_sales_csv():
    return """Date,UserID,OrderID,OrderValue
2023-07-15,user123,ord001,150.00
2023-07-15,user789,ord002,75.50
2023-07-16,user456,ord003,220.00
2023-07-16,user123,ord004,50.00
"""

# --- Load simulated CSVs into DataFrames ---
df_google_ads = pd.read_csv(io.StringIO(generate_google_ads_csv()), parse_dates=['Date'])
df_facebook_ads = pd.read_csv(io.StringIO(generate_facebook_ads_csv()), parse_dates=['Day'])
df_analytics = pd.read_csv(io.StringIO(generate_analytics_csv()), parse_dates=['Date'])
df_sales = pd.read_csv(io.StringIO(generate_sales_csv()), parse_dates=['Date'])

print("--- Data sources loaded successfully ---")


# Step 1: ETL - Cleaning and Standardizing Data
# -----------------------------------------------------------------------------
print("\n--- Step 1: Cleaning and Standardizing Data ---")

# Standardize Facebook Ads data
df_facebook_ads.rename(columns={'Day': 'Date', 'LinkClicks': 'Clicks', 'AmountSpent': 'Cost', 'AdSet': 'Campaign'}, inplace=True)
df_facebook_ads['Source'] = 'Facebook Ads'
df_facebook_ads['Impressions'] = df_facebook_ads['Reach'] # Assuming Reach is equivalent to Impressions

# Standardize Google Ads data
df_google_ads['Source'] = 'Google Ads'

# Combine ad spend data into a single DataFrame
df_ad_spend = pd.concat([
    df_google_ads[['Date', 'Impressions', 'Clicks', 'Cost', 'Source']],
    df_facebook_ads[['Date', 'Impressions', 'Clicks', 'Cost', 'Source']]
])

# Create a clear Marketing Channel from analytics data
source_mapping = {
    'google': 'Google Ads',
    'facebook': 'Facebook Ads',
    'promo_email': 'Email Marketing',
    '(direct)': 'Direct'
}
df_analytics['MarketingChannel'] = df_analytics['SessionSource'].map(source_mapping).fillna('Other')

print("Ad spend data unified and analytics channels mapped.")


# Step 2: Last-Touch Attribution Modeling
# -----------------------------------------------------------------------------
# This is the core logic: connecting sales to the last marketing touchpoint.
print("\n--- Step 2: Applying Last-Touch Attribution Model ---")

# Prepare interactions table, sorted by user and time
df_interactions = df_analytics[['Date', 'UserID', 'MarketingChannel']].sort_values(by=['UserID', 'Date'])

# Use pandas merge_asof to find the last interaction for each sale
# This is a highly efficient method for this type of time-series join
df_attributed_sales = pd.merge_asof(
    df_sales.sort_values('Date'),
    df_interactions.sort_values(['Date', 'UserID']), # Sort df_interactions by Date and UserID
    on='Date',
    by='UserID',
    direction='backward'  # Finds the last interaction on or before the sale date
)

print("Sales data attributed to marketing channels:")
print(df_attributed_sales)


# Step 3: Building the Final Performance Model
# -----------------------------------------------------------------------------
print("\n--- Step 3: Creating the Final Aggregated Performance Table ---")

# Aggregate costs by date and channel
df_daily_costs = df_ad_spend.groupby(['Date', 'Source'])['Cost'].sum().reset_index()

# Aggregate attributed revenue by date and channel
df_daily_revenue = df_attributed_sales.groupby(['Date', 'MarketingChannel'])['OrderValue'].sum().reset_index()
df_daily_revenue.rename(columns={'MarketingChannel': 'Source'}, inplace=True)

# Merge costs and revenue to create the final performance table
df_performance = pd.merge(
    df_daily_costs,
    df_daily_revenue,
    on=['Date', 'Source'],
    how='outer'  # Use outer join to see channels with cost but no revenue, or vice-versa
).fillna(0)

# Calculate final KPIs: ROI and Customer Acquisition Cost (CAC)
df_performance['ROI'] = (df_performance['OrderValue'] - df_performance['Cost']) / df_performance['Cost']
# Handle cases where cost is 0 to avoid infinite ROI
df_performance['ROI'] = df_performance['ROI'].replace([np.inf, -np.inf], 0)

print("Final performance table with ROI calculated:")
print(df_performance)


# Step 4: Loading to Warehouse and Visualization
# -----------------------------------------------------------------------------
print("\n--- Step 4: Visualizing the Marketing Funnel and ROI ---")

# --- Load to a simulated Data Warehouse (SQLite) ---
conn = sqlite3.connect('marketing_warehouse.db')
df_performance.to_sql('F_ChannelPerformance', conn, if_exists='replace', index=False)
# You would also load dimension tables for Channels, Dates, etc.
conn.close()

# --- Funnel Visualization ---
total_impressions = df_ad_spend['Impressions'].sum()
total_clicks = df_ad_spend['Clicks'].sum()
total_sessions = len(df_analytics)  # Approximation of website visits
total_purchases = len(df_sales['OrderID'].unique())

fig_funnel = go.Figure(go.Funnel(
    y=["Ad Impressions", "Ad Clicks", "Website Sessions", "Purchases"],
    x=[total_impressions, total_clicks, total_sessions, total_purchases],
    textposition="inside",
    textinfo="value+percent initial",
    marker={"color": ["#004c6d", "#487da8", "#82b2e0", "#c1eaff"]}
))
fig_funnel.update_layout(
    title_text="Multi-Channel Conversion Funnel",
    title_x=0.5
)
fig_funnel.show()

# --- ROI by Channel Visualization ---
df_channel_roi = df_performance.groupby('Source').agg(
    TotalCost=('Cost', 'sum'),
    TotalRevenue=('OrderValue', 'sum')
).reset_index()
df_channel_roi['ROI'] = (df_channel_roi['TotalRevenue'] - df_channel_roi['TotalCost']) / df_channel_roi['TotalCost']

fig_roi = px.bar(
    df_channel_roi.sort_values('ROI', ascending=False),
    x='Source',
    y='ROI',
    color='Source',
    title='Return on Investment (ROI) by Marketing Channel',
    labels={'ROI': 'Return on Investment (Ratio)', 'Source': 'Marketing Channel'},
    template='plotly_white'
)
fig_roi.update_layout(title_x=0.5)
fig_roi.show()

--- Data sources loaded successfully ---

--- Step 1: Cleaning and Standardizing Data ---
Ad spend data unified and analytics channels mapped.

--- Step 2: Applying Last-Touch Attribution Model ---
Sales data attributed to marketing channels:
        Date   UserID OrderID  OrderValue MarketingChannel
0 2023-07-15  user123  ord001       150.0       Google Ads
1 2023-07-15  user789  ord002        75.5           Direct
2 2023-07-16  user456  ord003       220.0       Google Ads
3 2023-07-16  user123  ord004        50.0  Email Marketing

--- Step 3: Creating the Final Aggregated Performance Table ---
Final performance table with ROI calculated:
        Date           Source    Cost  OrderValue       ROI
0 2023-07-15           Direct    0.00        75.5  0.000000
1 2023-07-15     Facebook Ads  580.00         0.0 -1.000000
2 2023-07-15       Google Ads  650.50       150.0 -0.769408
3 2023-07-16  Email Marketing    0.00        50.0  0.000000
4 2023-07-16     Facebook Ads  630.00         0.0 -1